#### in Windows Power Shell open elasticsearch folder (C:\Users\miria\elasticsearch-7.15.2) and type bin/elasticsearch to start the engine

## Libraries and Paths

In [1]:
from elasticsearch import Elasticsearch
import os
from os import listdir, mkdir, path
from PIL import Image

cwd = os.getcwd()
data_dir = cwd + "/images/"
print(data_dir)

d:\master_leipzig\Leipzig_S3\Information Retrieval\irlab/images/


# Sentiment Analysis

In [ ]:
import spacy
import pandas as pd
from afinn import Afinn
from os import listdir

nlp = spacy.load("en_core_web_sm")

afinn_df = pd.read_csv("data/AFINN-en-165.tsv", sep="\t", header=None, names=["word", "score"])
#afinn_df
afinn = Afinn()

vad_df = pd.read_csv("data/NRC-VAD-Lexicon.tsv", sep="\t")
#vad_df

In [ ]:
afinn_scores = dict(zip(afinn_df["word"], afinn_df["score"]))

def afinn_score(text):
    return sum(afinn_scores.get(t.lemma_, 0) for t in nlp(text))

word_valence = dict(zip(vad_df["Word"], vad_df["Valence"]))
word_arousal = dict(zip(vad_df["Word"], vad_df["Arousal"]))
word_dminanc = dict(zip(vad_df["Word"], vad_df["Dominance"]))

def text_valence(text, agg=sum):
    return agg([word_valence.get(t.lemma_, 0) for t in nlp(text)])

def text_arousal(text, agg=sum):
    return agg([word_arousal.get(t.lemma_, 0) for t in nlp(text)])

def text_dminanc(text, agg=sum):
    return agg([word_dminanc.get(t.lemma_, 0) for t in nlp(text)])

## Elastic Search: start and create Index

In [2]:
# Elastic search configuation
es = Elasticsearch(HOST="http://localhost", PORT=9200)
es = Elasticsearch()

In [3]:
# Creating index if it does not already exists
if not (es.indices.exists(index="webpages")):
    es.indices.create(index="webpages")
else:
    print("Index already exists")

Index already exists


C:\Users\miria\anaconda3\lib\site-packages\elasticsearch\connection\base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


## Add Documents to Index

In [32]:
# find all preprocessed text files and add them to index
image_files = [f for f in listdir(data_dir)]
for f in image_files:
    subdir = data_dir + f
    subdir_files = [f for f in listdir(subdir)]
    print("\n", f)

    for f2 in subdir_files:
        print("   ", subdir_files.index(f2), " von ", len(subdir_files))
        print(f2)
        subsubdir = subdir + "/" + f2 + "/pages/"
        subsubdir_files = [f for f in listdir(subsubdir)]

        for f3 in subsubdir_files:
            subsubsubdir = subsubdir + "/" + f3 + "/snapshot/"
            
            # open text file
            with open(subsubsubdir + 'text_pp.txt', errors='ignore') as fp:
                doc = fp.readlines()
                doc = ''.join(e for e in doc)
                doc_as_map = {"text":doc}

                es.index(index = "webpages", id = f2, document = doc_as_map)        

d:\master_leipzig\Leipzig_S3\Information Retrieval\irlab/images/

 I00
    0  von  99
I000330ba4ea0ad13
    1  von  99
I000d8de9c4746ee9
    2  von  99
I000f34bd3f8cb030
    3  von  99
I000f4116b93c5326
    4  von  99
I0010d5b5473065fa
    5  von  99
I001652ec040f07c4
    6  von  99
I001a318b3176d501
    7  von  99
I001b3068979b8629
    8  von  99
I001c2de5ef28f77f
    9  von  99
I001d567f0fb443d7
    10  von  99
I00246a6916fec768
    11  von  99
I0024c0097c425855
    12  von  99
I002747d3611b43da
    13  von  99
I0027bcd7cd6938d9
    14  von  99
I00290a8d0fe34013
    15  von  99
I002959a3418da38a
    16  von  99
I002d0f682f42a16d
    17  von  99
I002d16483c9a4e26
    18  von  99
I002dd0599859cb69
    19  von  99
I002e942703b3e786
    20  von  99
I002ead68c981102c
    21  von  99
I00316041c643e823
    22  von  99
I00369c26ceaba18c
    23  von  99
I003811155228eb5d
    24  von  99
I003a0ad4cc3bae04
    25  von  99
I003afc8037f37ff2
    26  von  99
I003cedcf82e2f803
    27  von  99
I003d

In [33]:
# get doc with certain ID (just to check)
res = es.get(index="webpages", id="I000330ba4ea0ad13")
res

## Make queries to the index

In [4]:
print('Please enter your query: ')
q = input()
print("Looking in the index for: " + q + " and showing the most relevent picture")

# make query

body = {
    "from":0,
    "size":50,
    "query": {
        "match": {
            "text":q
        }
    }
}
res = es.search(index="webpages", body=body)


# get ID's from retrieved documents

result_ids = []
results = res.get('hits').get('hits')

for doc in results:
    id = doc.get('_id')
    result_ids.append(id)


# create array with all image paths from id's

pictures = []
for i in range(len(result_ids)):
    picture_path = data_dir  + result_ids[i][0:3] + "/" + result_ids[i] + "/image.png"
    pictures.append(picture_path)
# print(pictures)


# show certain picture from pictures (default: first picture)

im = Image.open(pictures[0]) 
im.show()

Please enter your query: 
Looking in the index for: war and showing the most relevent picture


<ipython-input-4-ead1d2e96cfc>:16: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index="webpages", body=body)
